In [1]:
## Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
## Load the dataset
df = pd.read_csv("census-income.csv")
df.head()

,73,Not in universe,0,0.1,High school graduate,0.2,Not in universe.1,Widowed,Not in universe or children,Not in universe.2,...,United-States,United-States.1,United-States.2,Native- Born in the United States,0.7,Not in universe.8,2,0.8,95,-50000
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,-50000
1,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,-50000
2,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,-50000
3,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,-50000
4,48,Private,40,10,Some college but no degree,1200,Not in universe,Married-civilian spouse present,Entertainment,Professional specialty,...,Philippines,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,95,-50000


In [3]:
df.shape

(199522, 42)

## Global cleaning

In [4]:
cols = [f"feature_{i}" for i in range(41)] + ["income_label"]
df.columns = cols

In [5]:
columns = [
    "age",
    "class_of_worker",
    "detailed_industry_recode",
    "detailed_occupation_recode",
    "education",
    "wage_per_hour",
    "enroll_in_edu_last_week",
    "marital_status",
    "major_industry_code",
    "major_occupation_code",
    "race",
    "hispanic_origin",
    "sex",
    "member_of_labor_union",
    "reason_for_unemployment",
    "employment_status",
    "capital_gains",
    "capital_losses",
    "dividends_from_stocks",
    "tax_filer_status",
    "region_prev_residence",
    "state_prev_residence",
    "detailed_household_family_stat",
    "detailed_household_summary",
    "migration_change_msa",
    "migration_change_region",
    "migration_move_within_region",
    "live_in_house_1yr_ago",
    "migration_prev_res_sunbelt",
    "num_persons_worked_for_employer",
    "family_members_under_18",
    "country_birth_father",
    "country_birth_mother",
    "country_birth_self",
    "citizenship",
    "own_business_or_self_employed",
    "fill_income_questionnaire_veteran",
    "veterans_benefits",
    "weeks_worked_in_year",
    "year",
    "instance_weight",
    "income_label"
]


In [6]:
assert df.shape[1] == len(columns), "Column count mismatch"

df.columns = columns

In [7]:
df.head()

,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_last_week,marital_status,major_industry_code,major_occupation_code,...,country_birth_mother,country_birth_self,citizenship,own_business_or_self_employed,fill_income_questionnaire_veteran,veterans_benefits,weeks_worked_in_year,year,instance_weight,income_label
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,-50000
1,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,-50000
2,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,-50000
3,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,-50000
4,48,Private,40,10,Some college but no degree,1200,Not in universe,Married-civilian spouse present,Entertainment,Professional specialty,...,Philippines,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,95,-50000


In [8]:
## Create the binary target
df["target"] = (df["income_label"].str.contains("50000\\+")).astype(int)

In [9]:
df.shape

(199522, 43)

In [10]:
df.drop(['income_label'], axis = 1, inplace = True)
df.shape

(199522, 42)

In [11]:
x = df.drop(columns=["target", "instance_weight"])
y = df["target"].copy()

In [12]:
x.shape

(199522, 40)

In [13]:
y.shape

(199522,)

In [14]:
df.head()

,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_last_week,marital_status,major_industry_code,major_occupation_code,...,country_birth_mother,country_birth_self,citizenship,own_business_or_self_employed,fill_income_questionnaire_veteran,veterans_benefits,weeks_worked_in_year,year,instance_weight,target
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,0
1,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,0
2,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,0
3,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,0
4,48,Private,40,10,Some college but no degree,1200,Not in universe,Married-civilian spouse present,Entertainment,Professional specialty,...,Philippines,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,95,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199522 entries, 0 to 199521
Data columns (total 42 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   age                                199522 non-null  int64  
 1   class_of_worker                    199522 non-null  object 
 2   detailed_industry_recode           199522 non-null  int64  
 3   detailed_occupation_recode         199522 non-null  int64  
 4   education                          199522 non-null  object 
 5   wage_per_hour                      199522 non-null  int64  
 6   enroll_in_edu_last_week            199522 non-null  object 
 7   marital_status                     199522 non-null  object 
 8   major_industry_code                199522 non-null  object 
 9   major_occupation_code              199522 non-null  object 
 10  race                               199522 non-null  object 
 11  hispanic_origin                    1995

In [16]:
## missing valuues
miss = df.isna().sum().sort_values(ascending = False)
miss

age                                  0
country_birth_father                 0
detailed_household_summary           0
migration_change_msa                 0
migration_change_region              0
migration_move_within_region         0
live_in_house_1yr_ago                0
migration_prev_res_sunbelt           0
num_persons_worked_for_employer      0
family_members_under_18              0
country_birth_mother                 0
class_of_worker                      0
country_birth_self                   0
citizenship                          0
own_business_or_self_employed        0
fill_income_questionnaire_veteran    0
veterans_benefits                    0
weeks_worked_in_year                 0
year                                 0
instance_weight                      0
detailed_household_family_stat       0
state_prev_residence                 0
region_prev_residence                0
tax_filer_status                     0
detailed_industry_recode             0
detailed_occupation_recod

In [17]:
df["target"].value_counts(normalize=True)

target
0    0.937942
1    0.062058
Name: proportion, dtype: float64

In [18]:
df.describe()

,age,detailed_industry_recode,detailed_occupation_recode,wage_per_hour,capital_gains,capital_losses,dividends_from_stocks,migration_change_msa,family_members_under_18,fill_income_questionnaire_veteran,weeks_worked_in_year,year,instance_weight,target
count,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000,199522.000000
mean,34.494006,15.352397,11.306613,55.427186,434.721169,37.313975,197.530523,1740.380471,1.956190,0.175439,1.514830,23.175013,94.499669,0.062058
std,22.310785,18.067141,14.454218,274.897115,4697.542951,271.897097,1984.168581,993.770642,2.365127,0.553696,0.851475,24.411494,0.500001,0.241262
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.870000,0.000000,0.000000,0.000000,0.000000,94.000000,0.000000
25%,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1061.607500,0.000000,0.000000,2.000000,0.000000,94.000000,0.000000
50%,33.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1618.310000,1.000000,0.000000,2.000000,8.000000,94.000000,0.000000
75%,50.000000,33.000000,26.000000,0.000000,0.000000,0.000000,0.000000,2188.610000,4.000000,0.000000,2.000000,52.000000,95.000000,0.000000
max,90.000000,51.000000,46.000000,9999.000000,99999.000000,4608.000000,99999.000000,18656.300000,6.000000,2.000000,2.000000,52.000000,95.000000,1.000000


In [19]:
df.describe(include = 'object')

,class_of_worker,education,enroll_in_edu_last_week,marital_status,major_industry_code,major_occupation_code,race,hispanic_origin,sex,member_of_labor_union,...,migration_move_within_region,live_in_house_1yr_ago,migration_prev_res_sunbelt,num_persons_worked_for_employer,country_birth_father,country_birth_mother,country_birth_self,citizenship,own_business_or_self_employed,veterans_benefits
count,199522,199522,199522,199522,199522,199522,199522,199522,199522,199522,...,199522,199522,199522,199522,199522,199522,199522,199522,199522,199522
unique,9,17,3,7,24,15,5,10,2,3,...,9,10,3,4,5,43,43,43,5,3
top,Not in universe,High school graduate,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,...,?,?,Not in universe under 1 year old,?,Not in universe,United-States,United-States,United-States,Native- Born in the United States,Not in universe
freq,100244,48406,186942,86485,100683,100683,167364,171906,103983,180458,...,99695,99695,101211,99695,144231,159162,160478,176988,176991,197538


In [20]:
df.select_dtypes(include = 'object').nunique()

class_of_worker                     9
education                          17
enroll_in_edu_last_week             3
marital_status                      7
major_industry_code                24
major_occupation_code              15
race                                5
hispanic_origin                    10
sex                                 2
member_of_labor_union               3
reason_for_unemployment             6
employment_status                   8
tax_filer_status                    6
region_prev_residence               6
state_prev_residence               51
detailed_household_family_stat     38
detailed_household_summary          8
migration_change_region            10
migration_move_within_region        9
live_in_house_1yr_ago              10
migration_prev_res_sunbelt          3
num_persons_worked_for_employer     4
country_birth_father                5
country_birth_mother               43
country_birth_self                 43
citizenship                        43
own_business

In [21]:
# Create a stable time index
df = df.reset_index(drop=True)
df["time_index"] = df["year"].astype(int) * 10**6 + df.index
df = df.sort_values("time_index").reset_index(drop=True)

# Split last 20% as production
n = len(df)
prod_start = int(n * 0.80)

df_train_full = df.iloc[:prod_start].copy()
df_prod = df.iloc[prod_start:].copy()

# Sanity check
print(df_train_full.shape, df_prod.shape)

(159617, 43) (39905, 43)


In [22]:
# Number of rows in training history
n_train = len(df_train_full)

# Define split points
s1 = int(n_train * 0.33)
s2 = int(n_train * 0.66)

# Create time-based slices
df_model1 = df_train_full.iloc[ : s1 ].copy()
df_model2 = df_train_full.iloc[ s1 : s2 ].copy()
df_model3 = df_train_full.iloc[s2 : ].copy()

# Sanity check
print(df_model1.shape, df_model2.shape, df_model3.shape)

(52673, 43) (52674, 43) (54270, 43)


In [23]:
df_prod.to_csv("census_prod.csv", index=False)

## Building model 1

In [24]:
# Separate features and target for Model 1
x1 = df_model1.drop(columns=["target", "time_index"])
y1 = df_model1["target"].copy()

In [25]:
# Identify numeric and categorical columns
num_cols1 = x1.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols1 = x1.select_dtypes(include=["object"]).columns.tolist()

In [26]:
## check skewness
from scipy.stats import skew
skewness1 = x1[num_cols1].apply(lambda x: skew(x.dropna()))
skewness1.sort_values(ascending=False)

capital_gains                        53.814169
dividends_from_stocks                26.623780
wage_per_hour                        23.253080
capital_losses                       17.298094
detailed_occupation_recode            5.672070
detailed_industry_recode              5.646099
migration_change_msa                  1.436528
age                                   0.687624
instance_weight                       0.004367
weeks_worked_in_year                 -0.002278
family_members_under_18                    NaN
fill_income_questionnaire_veteran          NaN
year                                       NaN
dtype: float64

In [27]:
pos_skew_cols1 = skewness1[skewness1 > 1].index.tolist()
neg_skew_cols1 = skewness1[skewness1 < -1].index.tolist()

In [28]:
for col in pos_skew_cols1:
    x1[col] = np.log1p(x1[col])

In [29]:
for col in neg_skew_cols1:
    x1[col] = np.log1p(x1[col].max() + 1 - x1[col])

In [30]:
## CORRELATION TREATMENT 1
corr_matrix1 = x1[num_cols1].corr().abs()

upper1 = corr_matrix1.where(np.triu(np.ones(corr_matrix1.shape), k=1).astype(bool))
to_drop1 = [column for column in upper1.columns if any(upper1[column] > 0.90)]

print(f"Dropping {len(to_drop1)} redundant features due to >0.9 correlation")
x1 = x1.drop(columns=to_drop1)
num_cols1 = [c for c in num_cols1 if c not in to_drop1]

Dropping 1 redundant features due to >0.9 correlation


In [31]:
split_m1 = int(len(x1) * 0.80)

x_train1 = x1.iloc[:split_m1]
x_test1   = x1.iloc[split_m1:]

y_train1 = y1.iloc[:split_m1]
y_test1   = y1.iloc[split_m1:]

In [32]:
low1, high1 = x_train1[col].quantile([0.01, 0.99])
x_train1[col] = x_train1[col].clip(low1, high1)
x_test1[col] = x_test1[col].clip(low1, high1)

In [33]:
np.isinf(x_train1[num_cols1]).sum().sort_values(ascending = False)

age                                  0
detailed_industry_recode             0
wage_per_hour                        0
capital_gains                        0
capital_losses                       0
dividends_from_stocks                0
migration_change_msa                 0
family_members_under_18              0
fill_income_questionnaire_veteran    0
weeks_worked_in_year                 0
year                                 0
instance_weight                      0
dtype: int64

In [34]:
x_train1[num_cols1] = x_train1[num_cols1].replace([np.inf, -np.inf], np.nan)
x_test1[num_cols1] = x_test1[num_cols1].replace([np.inf, -np.inf], np.nan)

In [35]:
x_train1[num_cols1] = x_train1[num_cols1].fillna(x_train1[num_cols1].median())
x_test1[num_cols1] = x_test1[num_cols1].fillna(x_train1[num_cols1].median())

In [36]:
## scaling the numerical values
scaler_1 = StandardScaler()

x_train1_num_scaled = scaler_1.fit_transform(x_train1[num_cols1])
x_test1_num_scaled = scaler_1.transform(x_test1[num_cols1])

In [37]:
x_train1_num_df = pd.DataFrame(
    x_train1_num_scaled, 
    columns=num_cols1, 
    index=x_train1.index
)

x_test1_num_df = pd.DataFrame(
    x_test1_num_scaled, 
    columns=num_cols1, 
    index=x_test1.index
)

In [38]:
## encoding categorical columns

encoder1 = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
encoded_train1 = encoder1.fit_transform(x_train1[cat_cols1])
encoded_test1 = encoder1.transform(x_test1[cat_cols1])

In [39]:
## converting enocded features into a dataframe
encoded_train_df1 = pd.DataFrame(
    encoded_train1,
    columns=encoder1.get_feature_names_out(cat_cols1),
    index=x_train1.index
)

encoded_test_df1 = pd.DataFrame(
    encoded_test1,
    columns=encoder1.get_feature_names_out(cat_cols1),
    index=x_test1.index
)

In [40]:
## concatenating the categorical and numerical values
x_train1_final = pd.concat(
    [x_train1_num_df, encoded_train_df1],
    axis=1
)

x_test1_final = pd.concat(
    [x_test1_num_df, encoded_test_df1],
    axis=1
)

In [41]:
## Building the 1st Logisticc regression model

log_reg1 = LogisticRegression(max_iter=1000, class_weight='balanced')
logreg_model_1 = log_reg1.fit(x_train1_final, y_train1)

In [42]:
## Building the 1st Decision tree model

param_grid1 = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_leaf': [40, 80, 120],
    'min_samples_split': [3, 12, 25]
}

dt1 = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced'
)

grid1 = GridSearchCV(
    estimator=dt1,
    param_grid=param_grid1,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_model1 = grid1.fit(x_train1_final, y_train1)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


In [43]:
print("Best parameters:", grid1.best_params_)
print("Best CV ROC-AUC:", grid1.best_score_)

Best parameters: {'max_depth': 3, 'min_samples_leaf': 80, 'min_samples_split': 3}
Best CV ROC-AUC: 0.9221373266372045


In [44]:
best_tree1 = DecisionTreeClassifier(
    **grid1.best_params_,
    random_state=42,
    class_weight='balanced'
)

dt_model_1 = best_tree1.fit(x_train1_final, y_train1);

In [45]:
##Model evaluation

def evaluate_model(model, X, y):
    probs = model.predict_proba(X)[:, 1]
    return {
        "ROC_AUC": roc_auc_score(y, probs),
        "PR_AUC": average_precision_score(y, probs)
    }

logreg_metrics1 = evaluate_model(logreg_model_1, x_test1_final, y_test1)
tree_metrics1 = evaluate_model(dt_model_1, x_test1_final, y_test1)

print("Logistic Regression1:", logreg_metrics1)
print("Decision Tree1:", tree_metrics1)

Logistic Regression1: {'ROC_AUC': 0.9451723787434971, 'PR_AUC': 0.1575888348797036}
Decision Tree1: {'ROC_AUC': 0.933991554319402, 'PR_AUC': 0.1025294424275367}


In [46]:
## selecting the 1st best model
best_model1 = log_reg1 if logreg_metrics1['PR_AUC'] >= tree_metrics1['PR_AUC'] else best_tree1
print(best_model1)

LogisticRegression(class_weight='balanced', max_iter=1000)


In [47]:
# Combine train + test
x_full1 = pd.concat([x_train1_final, x_test1_final])
y_full1 = pd.concat([y_train1, y_test1])

# Refit the selected model
best_model1.fit(x_full1, y_full1)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [48]:
## Saving the trained model.
import joblib

joblib.dump(best_model1, "census_model_1.pkl")

['census_model_1.pkl']

In [49]:
## freezing the preprocessing

joblib.dump(scaler_1, "census_model_1_scaler.pkl")
joblib.dump(encoder1, "census_model_1_encoder.pkl")

['census_model_1_encoder.pkl']

In [50]:
import json
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score


y_full1_proba = best_model1.predict_proba(x_full1)[:, 1]
y_full1_pred = best_model1.predict(x_full1)


roc_auc_full1 = roc_auc_score(y_full1, y_full1_proba)
pr_auc_full1 = average_precision_score(y_full1, y_full1_proba)


accuracy_full1 = accuracy_score(y_full1, y_full1_pred)
positive_rate_full1 = float(np.mean(y_full1_pred))


prediction_stats1 = {
    "mean_proba": float(np.mean(y_full1_proba)),
    "std_proba": float(np.std(y_full1_proba)),
    "p95_proba": float(np.percentile(y_full1_proba, 95))
}

x_full_num1_scaled = scaler_1.transform(x1[num_cols1])
x_full_num1_scaled_df = pd.DataFrame(x_full_num1_scaled, columns=num_cols1)


feature_means1 = x_full_num1_scaled_df.mean().to_dict()
feature_stds1 = x_full_num1_scaled_df.std().to_dict()

feature_means1 = {k: float(v) for k, v in feature_means1.items()}
feature_stds1 = {k: float(v) for k, v in feature_stds1.items()}


baseline_model_1 = {
    "model_version": "census_model_1",
    "dataset": "census",
    "model_type": type(best_model1).__name__,

    "primary_metrics": {
        "roc_auc": float(roc_auc_full1),
        "pr_auc": float(pr_auc_full1)
    },

    "secondary_metrics": {
        "accuracy": float(accuracy_full1),
        "positive_rate": positive_rate_full1
    },

    "prediction_stats": prediction_stats1,

    "feature_means": feature_means1,
    "feature_stds": feature_stds1
}

with open("census_model_1_baseline.json", "w") as f:
    json.dump(baseline_model_1, f, indent=4)

In [51]:
preprocessing_data1 = {
    "num_cols": num_cols1,
    "cat_cols": cat_cols1,
    "pos_skew_cols": pos_skew_cols1,
    "neg_skew_cols": neg_skew_cols1
}

joblib.dump(preprocessing_data1, "census_model_1_preprocessing.pkl")

['census_model_1_preprocessing.pkl']

## Building model II

In [52]:
# Separate features and target for Model 2
x2 = df_model2.drop(columns=["target", "time_index"])
y2 = df_model2["target"].copy()

In [53]:
# Identify numeric and categorical columns
num_cols2 = x2.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols2 = x2.select_dtypes(include=["object"]).columns.tolist()

In [54]:
## check skewness
from scipy.stats import skew
skewness2 = x2[num_cols2].apply(lambda x: skew(x.dropna()))
skewness2.sort_values(ascending=False)

capital_gains                        41.028886
dividends_from_stocks                27.789652
wage_per_hour                        23.114080
capital_losses                       13.471337
fill_income_questionnaire_veteran     6.388988
detailed_occupation_recode            2.716517
family_members_under_18               2.644007
detailed_industry_recode              2.612670
year                                  2.416902
migration_change_msa                  1.528820
age                                   0.699682
instance_weight                       0.016024
weeks_worked_in_year                 -0.355229
dtype: float64

In [55]:
pos_skew_cols2 = skewness2[skewness2 > 1].index.tolist()
neg_skew_cols2 = skewness2[skewness2 < -1].index.tolist()

In [56]:
for col in pos_skew_cols2:
    x2[col] = np.log1p(x2[col])

In [57]:
for col in neg_skew_cols2:
    x2[col] = np.log1p(x2[col].max() + 1 - x2[col])

In [58]:
## CORRELATION TREATMENT 2
corr_matrix2 = x2[num_cols2].corr().abs()

upper2 = corr_matrix2.where(np.triu(np.ones(corr_matrix2.shape), k=1).astype(bool))
to_drop2 = [column for column in upper2.columns if any(upper2[column] > 0.90)]

print(f"Dropping {len(to_drop2)} redundant features due to >0.9 correlation")
x2 = x2.drop(columns=to_drop2)
num_cols2 = [c for c in num_cols2 if c not in to_drop2]

Dropping 1 redundant features due to >0.9 correlation


In [59]:
split_m2 = int(len(x2) * 0.80)

x_train2 = x2.iloc[ : split_m2 ]
x_test2   = x2.iloc[ split_m2 : ]

y_train2 = y2.iloc[ : split_m2 ]
y_test2   = y2.iloc[ split_m2 : ]

In [60]:
low2, high2 = x_train2[col].quantile([0.01, 0.99])
x_train2[col] = x_train2[col].clip(low2, high2)
x_test2[col] = x_test2[col].clip(low2, high2)

In [61]:
np.isinf(x_train2[num_cols2]).sum().sort_values(ascending = False)

age                                  0
detailed_industry_recode             0
wage_per_hour                        0
capital_gains                        0
capital_losses                       0
dividends_from_stocks                0
migration_change_msa                 0
family_members_under_18              0
fill_income_questionnaire_veteran    0
weeks_worked_in_year                 0
year                                 0
instance_weight                      0
dtype: int64

In [62]:
x_train2[num_cols2] = x_train2[num_cols2].replace([np.inf, -np.inf], np.nan)
x_test2[num_cols2] = x_test2[num_cols2].replace([np.inf, -np.inf], np.nan)

In [63]:
x_train2[num_cols2] = x_train2[num_cols2].fillna(x_train2[num_cols2].median())
x_test2[num_cols2] = x_test2[num_cols2].fillna(x_train2[num_cols2].median())

In [64]:
## scaling the numerical values
scaler_2 = StandardScaler()

x_train2_num_scaled = scaler_2.fit_transform(x_train2[num_cols2])
x_test2_num_scaled = scaler_2.transform(x_test2[num_cols2])

In [65]:
x_train2_num_df = pd.DataFrame(
    x_train2_num_scaled, 
    columns=num_cols2, 
    index=x_train2.index
)

x_test2_num_df = pd.DataFrame(
    x_test2_num_scaled, 
    columns=num_cols2, 
    index=x_test2.index
)

In [66]:
## encoding categorical columns

encoder2 = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
encoded_train2 = encoder2.fit_transform(x_train2[cat_cols2])
encoded_test2 = encoder2.transform(x_test2[cat_cols2])

In [67]:
## converting enocded features into a dataframe
encoded_train_df2 = pd.DataFrame(
    encoded_train2,
    columns=encoder2.get_feature_names_out(cat_cols2),
    index=x_train2.index
)

encoded_test_df2 = pd.DataFrame(
    encoded_test2,
    columns=encoder2.get_feature_names_out(cat_cols2),
    index=x_test2.index
)

In [68]:
## concatenating the categorical and numerical values
x_train2_final = pd.concat(
    [x_train2_num_df, encoded_train_df2],
    axis=1
)

x_test2_final = pd.concat(
    [x_test2_num_df, encoded_test_df2],
    axis=1
)

In [69]:
## Building the 2nd Logisticc regression model

log_reg2 = LogisticRegression(max_iter=1000, class_weight='balanced')
logreg_model_2 = log_reg2.fit(x_train2_final, y_train2)

In [70]:
## Building the 2nd Decision tree model


param_grid2 = {
    'max_depth': [5, 8, 11, 14],
    'min_samples_leaf': [30, 70, 110],
    'min_samples_split': [4, 13, 23]
}



dt2 = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced'
)

grid2 = GridSearchCV(
    estimator=dt2,
    param_grid=param_grid2,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)


grid_model2 = grid2.fit(x_train2_final, y_train2)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


In [71]:
print("Best parameters:", grid2.best_params_)
print("Best CV ROC-AUC:", grid2.best_score_)

Best parameters: {'max_depth': 5, 'min_samples_leaf': 30, 'min_samples_split': 4}
Best CV ROC-AUC: 0.9183697800137165


In [72]:
best_tree2 = DecisionTreeClassifier(
    **grid2.best_params_,
    random_state=42,
    class_weight='balanced'
)

dt_model_2 = best_tree2.fit(x_train2_final, y_train2);

In [73]:
##Model evaluation

logreg_metrics2 = evaluate_model(logreg_model_2, x_test2_final, y_test2)
tree_metrics2 = evaluate_model(dt_model_2, x_test2_final, y_test2)

print("Logistic Regression2:", logreg_metrics2)
print("Decision Tree2:", tree_metrics2)

Logistic Regression2: {'ROC_AUC': 0.8416705702892284, 'PR_AUC': 0.11860677569934583}
Decision Tree2: {'ROC_AUC': 0.7479836288075334, 'PR_AUC': 0.14073810110569168}


In [74]:
## selecting the 2nd best model
best_model2 = log_reg2 if logreg_metrics2['PR_AUC'] >= tree_metrics2['PR_AUC'] else best_tree2
print(best_model2)

DecisionTreeClassifier(class_weight='balanced', max_depth=5,
                       min_samples_leaf=30, min_samples_split=4,
                       random_state=42)


In [75]:
# Combine train + test
x_full2 = pd.concat([x_train2_final, x_test2_final])
y_full2 = pd.concat([y_train2, y_test2])

# Refit the selected model
best_model2.fit(x_full2, y_full2)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,4
,min_samples_leaf,30
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,'balanced'


In [76]:
## Saving the 1st trained model.
import joblib

joblib.dump(best_model2, "census_model_2.pkl")

['census_model_2.pkl']

In [77]:
## freezing the preprocessing

joblib.dump(scaler_2, "census_model_2_scaler.pkl")
joblib.dump(encoder2, "census_model_2_encoder.pkl")

['census_model_2_encoder.pkl']

In [78]:
preprocessing_data2 = {
    "num_cols": num_cols2,
    "cat_cols": cat_cols2,
    "pos_skew_cols": pos_skew_cols2,
    "neg_skew_cols": neg_skew_cols2
}

joblib.dump(preprocessing_data2, "census_model_2_preprocessing.pkl")

['census_model_2_preprocessing.pkl']

In [79]:
import json
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score


y_full2_proba = best_model2.predict_proba(x_full2)[:, 1]
y_full2_pred = best_model2.predict(x_full2)


roc_auc_full2 = roc_auc_score(y_full2, y_full2_proba)
pr_auc_full2 = average_precision_score(y_full2, y_full2_proba)


accuracy_full2 = accuracy_score(y_full2, y_full2_pred)
positive_rate_full2 = float(np.mean(y_full2_pred))


prediction_stats2 = {
    "mean_proba": float(np.mean(y_full2_proba)),
    "std_proba": float(np.std(y_full2_proba)),
    "p95_proba": float(np.percentile(y_full2_proba, 95))
}


x_full_num2_scaled = scaler_2.transform(x2[num_cols2])
x_full_num2_scaled_df = pd.DataFrame(x_full_num2_scaled, columns=num_cols2)


feature_means2 = x_full_num2_scaled_df.mean().to_dict()
feature_stds2 = x_full_num2_scaled_df.std().to_dict()

feature_means2 = {k: float(v) for k, v in feature_means2.items()}
feature_stds2 = {k: float(v) for k, v in feature_stds2.items()}


baseline_model_2 = {
    "model_version": "census_model_2",
    "dataset": "census",
    "model_type": type(best_model2).__name__,

    "primary_metrics": {
        "roc_auc": float(roc_auc_full2),
        "pr_auc": float(pr_auc_full2)
    },

    "secondary_metrics": {
        "accuracy": float(accuracy_full2),
        "positive_rate": positive_rate_full2
    },

    "prediction_stats": prediction_stats2,

    "feature_means": feature_means2,
    "feature_stds": feature_stds2
}

with open("census_model_2_baseline.json", "w") as f:
    json.dump(baseline_model_2, f, indent=4)

## Building model III

In [80]:
# Separate features and target for Model 3
x3 = df_model3.drop(columns=["target", "time_index"])
y3 = df_model3["target"].copy()

In [81]:
# Identify numeric and categorical columns
num_cols3 = x3.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols3 = x3.select_dtypes(include=["object"]).columns.tolist()

In [82]:
## check skewness
from scipy.stats import skew
skewness3 = x3[num_cols3].apply(lambda x: skew(x.dropna()))
skewness3.sort_values(ascending=False)

dividends_from_stocks                29.051557
capital_gains                        14.954174
wage_per_hour                         6.477600
capital_losses                        5.643566
fill_income_questionnaire_veteran     1.500193
migration_change_msa                  1.376528
age                                   0.479376
instance_weight                      -0.006044
family_members_under_18              -0.142085
detailed_occupation_recode           -0.198742
detailed_industry_recode             -0.801927
year                                 -1.321625
weeks_worked_in_year                -11.262340
dtype: float64

In [83]:
pos_skew_cols3 = skewness3[skewness3 > 1].index.tolist()
neg_skew_cols3 = skewness3[skewness3 < -1].index.tolist()

In [84]:
for col in pos_skew_cols3:
    x3[col] = np.log1p(x3[col])

In [85]:
## CORRELATION TREATMENT 3
corr_matrix3 = x3[num_cols3].corr().abs()

upper3 = corr_matrix3.where(np.triu(np.ones(corr_matrix3.shape), k=1).astype(bool))
to_drop3 = [column for column in upper3.columns if any(upper3[column] > 0.90)]

print(f"Dropping {len(to_drop3)} redundant features due to >0.9 correlation")
x3 = x3.drop(columns=to_drop3)
num_cols3 = [c for c in num_cols3 if c not in to_drop3]

Dropping 0 redundant features due to >0.9 correlation


In [86]:
split_m3 = int(len(x3) * 0.80)

x_train3 = x3.iloc[ : split_m3 ]
x_test3   = x3.iloc[ split_m3 : ]

y_train3 = y3.iloc[ : split_m3 ]
y_test3   = y3.iloc[ split_m3 : ]

In [87]:
low3, high3 = x_train3[col].quantile([0.01, 0.99])
x_train3[col] = x_train3[col].clip(low3, high3)
x_test3[col] = x_test3[col].clip(low3, high3)

In [88]:
np.isinf(x_train3[num_cols3]).sum().sort_values(ascending = False)

age                                  0
detailed_industry_recode             0
detailed_occupation_recode           0
wage_per_hour                        0
capital_gains                        0
capital_losses                       0
dividends_from_stocks                0
migration_change_msa                 0
family_members_under_18              0
fill_income_questionnaire_veteran    0
weeks_worked_in_year                 0
year                                 0
instance_weight                      0
dtype: int64

In [89]:
x_train3[num_cols3] = x_train3[num_cols3].replace([np.inf, -np.inf], np.nan)
x_test3[num_cols3] = x_test3[num_cols3].replace([np.inf, -np.inf], np.nan)

In [90]:
x_train3[num_cols3] = x_train3[num_cols3].fillna(x_train3[num_cols3].median())
x_test3[num_cols3] = x_test3[num_cols3].fillna(x_train3[num_cols3].median())

In [91]:
## scaling the numerical values
scaler_3 = StandardScaler()

x_train3_num_scaled = scaler_3.fit_transform(x_train3[num_cols3])
x_test3_num_scaled = scaler_3.transform(x_test3[num_cols3])

In [92]:
x_train3_num_df = pd.DataFrame(
    x_train3_num_scaled, 
    columns=num_cols3, 
    index=x_train3.index
)

x_test3_num_df = pd.DataFrame(
    x_test3_num_scaled, 
    columns=num_cols3, 
    index=x_test3.index
)

In [93]:
## encoding categorical columns

encoder3 = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
encoded_train3 = encoder3.fit_transform(x_train3[cat_cols3])
encoded_test3 = encoder3.transform(x_test3[cat_cols3])

In [94]:
## converting enocded features into a dataframe
encoded_train_df3 = pd.DataFrame(
    encoded_train3,
    columns=encoder3.get_feature_names_out(cat_cols3),
    index=x_train3.index
)

encoded_test_df3 = pd.DataFrame(
    encoded_test3,
    columns=encoder3.get_feature_names_out(cat_cols3),
    index=x_test3.index
)

In [95]:
## concatenating the categorical and numerical values
x_train3_final = pd.concat(
    [x_train3_num_df, encoded_train_df3],
    axis=1
)

x_test3_final = pd.concat(
    [x_test3_num_df, encoded_test_df3],
    axis=1
)

In [96]:
## Building the 3rd Logisticc regression model

log_reg3 = LogisticRegression(max_iter=1000, class_weight='balanced')
logreg_model_3 = log_reg3.fit(x_train3_final, y_train3)

In [97]:
## Building the 3rd Decision tree model


param_grid3 = {
    'max_depth': [5, 8, 11, 14],
    'min_samples_leaf': [30, 70, 110],
    'min_samples_split': [4, 13, 23]
}



dt3 = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced'
)

grid3 = GridSearchCV(
    estimator=dt3,
    param_grid=param_grid3,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)


grid_model3 = grid3.fit(x_train3_final, y_train3)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


In [98]:
print("Best parameters:", grid3.best_params_)
print("Best CV ROC-AUC:", grid3.best_score_)

Best parameters: {'max_depth': 8, 'min_samples_leaf': 110, 'min_samples_split': 4}
Best CV ROC-AUC: 0.6417606615708045


In [99]:
best_tree3 = DecisionTreeClassifier(
    **grid3.best_params_,
    random_state=42,
    class_weight='balanced'
)

dt_model_3 = best_tree3.fit(x_train3_final, y_train3);

In [100]:
##Model evaluation

logreg_metrics3 = evaluate_model(logreg_model_3, x_test3_final, y_test3)
tree_metrics3 = evaluate_model(dt_model_3, x_test3_final, y_test3)

print("Logistic Regression3:", logreg_metrics3)
print("Decision Tree3:", tree_metrics3)

Logistic Regression3: {'ROC_AUC': 0.8866697782507716, 'PR_AUC': 0.6192375021882034}
Decision Tree3: {'ROC_AUC': 0.8544267579923913, 'PR_AUC': 0.5226395090938075}


In [101]:
## selecting the best model
best_model3 = log_reg3 if logreg_metrics3['PR_AUC'] >= tree_metrics3['PR_AUC'] else best_tree3
print(best_model3)

LogisticRegression(class_weight='balanced', max_iter=1000)


In [102]:
# Combine train + test
x_full3 = pd.concat([x_train3_final, x_test3_final])
y_full3 = pd.concat([y_train3, y_test3])

# Refit the selected model
best_model3.fit(x_full3, y_full3)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [103]:
## Saving the trained model.
import joblib

joblib.dump(best_model3, "census_model_3.pkl")

['census_model_3.pkl']

In [104]:
## freezing the preprocessing

joblib.dump(scaler_3, "census_model_3_scaler.pkl")
joblib.dump(encoder3, "census_model_3_encoder.pkl")

['census_model_3_encoder.pkl']

In [105]:
preprocessing_data3 = {
    "num_cols": num_cols3,
    "cat_cols": cat_cols3,
    "pos_skew_cols": pos_skew_cols3,
    "neg_skew_cols": neg_skew_cols3
}

joblib.dump(preprocessing_data3, "census_model_3_preprocessing.pkl")

['census_model_3_preprocessing.pkl']

In [106]:
## import json
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score


y_full3_proba = best_model3.predict_proba(x_full3)[:, 1]
y_full3_pred = best_model3.predict(x_full3)


roc_auc_full3 = roc_auc_score(y_full3, y_full3_proba)
pr_auc_full3 = average_precision_score(y_full3, y_full3_proba)


accuracy_full3 = accuracy_score(y_full3, y_full3_pred)
positive_rate_full3 = float(np.mean(y_full3_pred))


prediction_stats3 = {
    "mean_proba": float(np.mean(y_full3_proba)),
    "std_proba": float(np.std(y_full3_proba)),
    "p95_proba": float(np.percentile(y_full3_proba, 95))
}


x_full_num3_scaled = scaler_3.transform(x3[num_cols3])
x_full_num3_scaled_df = pd.DataFrame(x_full_num3_scaled, columns=num_cols3)


feature_means3 = x_full_num3_scaled_df.mean().to_dict()
feature_stds3 = x_full_num3_scaled_df.std().to_dict()

feature_means3 = {k: float(v) for k, v in feature_means3.items()}
feature_stds3 = {k: float(v) for k, v in feature_stds3.items()}

baseline_model_3 = {
    "model_version": "census_model_3",
    "dataset": "census",
    "model_type": type(best_model3).__name__,

    "primary_metrics": {
        "roc_auc": float(roc_auc_full3),
        "pr_auc": float(pr_auc_full3)
    },

    "secondary_metrics": {
        "accuracy": float(accuracy_full3),
        "positive_rate": positive_rate_full3
    },

    "prediction_stats": prediction_stats3,

    "feature_means": feature_means3,
    "feature_stds": feature_stds3
}

with open("census_model_3_baseline.json", "w") as f:
    json.dump(baseline_model_3, f, indent=4)

In [107]:
x1.shape, x2.shape, x3.shape

((52673, 40), (52674, 40), (54270, 41))